In [1]:
!pip install xlrd 
!pip install openpyxl
!pip install msoffcrypto-tool
!pip install -q optuna

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

import optuna

# Data searching 

## Lists_on

In [ ]:
years = [i for i in range(2016, 2023)]
quarters_dict = {
    2016: [i for i in range(1, 5)],
    2017: [i for i in range(1, 5)],
    2018: [i for i in range(1, 5)],
    2019: [i for i in range(1, 5)],
    2020: [i for i in range(1, 5)],
    2021: [i for i in range(1, 5)],
    2022: [i for i in range(1, 4)],
    2023: [i for i in range(1, 3)],

}
df_full = pd.DataFrame()
         
for year in years:
    for n_quarter in quarters_dict[year]:
        otstup = 5
        if year == 2022 and n_quarter == 4 or year >= 2023:
            otstup = 4
            
        path = f"data/Lists ON/List ON_{year}/{n_quarter} quarter {year}.xls" 
        df = pd.read_excel(path)[otstup:]

        new_col = df.iloc[1].combine_first(df.iloc[0])
        new_col = list(new_col.values)

        df = df[3:]
        df.columns = new_col
        df = df.reset_index(drop=True)

        df['Год'] = np.full((df.shape[0],), year)
        df['Квартал'] = np.full((df.shape[0],), n_quarter)
        df['Отделение'] = np.full((df.shape[0],), 'Незвестное отделение')

        n = df.shape[0]
        name = ""
        ind_start = -1
        ind_end = -1
        
        for i, value in enumerate(df['№ п/п']):
            if type(value) == str and ('Отделение' in value or 'Дальне' in value):
                if ind_start == -1:
                    ind_start = df.loc[df['№ п/п'] == value].index[0]
                    name = " ".join(value.split(" ")[1:])
                else:
                    ind_end = df.loc[df['№ п/п'] == value].index[0]
                    df.loc[ind_start:ind_end, 'Отделение'] = name
                    df = df.drop(ind_start)
                    ind_start = ind_end
                    name = " ".join(value.split(" ")[1:])
            if i == n - 1:
                df.loc[ind_end:, 'Отделение'] = name 
         
        if year == 2016 and n_quarter == 1:
            df_full = df
        else:
#             print(year, n_quarter)
#             print(df.index)
#             print(df_full.index)
            if year >= 2023:
                df = df.iloc[:,1:]
                df = df.reset_index(drop=True)

                
            df_full = pd.concat([df_full, df], ignore_index = True)
            df_full = df_full.reset_index(drop=True)

df_full = df_full.reset_index(drop=True)

In [ ]:
df_full = df_full.iloc[:,1:]
df_full

In [ ]:
df_full['Техническое состояние'].unique()

In [ ]:
df_full.to_csv('Lists_on_2016_2022_all_quarters.csv', index=False)

## KC TR KR

In [ ]:
file_dict = {
    'KS': [_ for _ in range(2017, 2024)],
    "TR": [_ for _ in range(2019, 2024)],
    "KR": [_ for _ in range(2017, 2024)],
}
files_lst = os.listdir("data/KC_TR_KR")
df_ks_full = pd.DataFrame()
df_kr_full = pd.DataFrame()

for file_type in file_dict:
    for year in file_dict[file_type]:
        
        path = f"data/KC_TR_KR/{file_type} on 01.01.{year}.csv"
        
        df = pd.read_csv(path)
        if file_type == "KS":
            if year <= 2019:
                df = df[3:]
                new_col = df.iloc[1].combine_first(df.iloc[0])
                new_col = df.iloc[2].combine_first(new_col)
                new_col = df.iloc[3].combine_first(new_col)
                new_col = list(new_col.values)
                df = df[7:]
                df.columns = new_col
            else:
                df = df[5:]
                new_col = df.iloc[1].combine_first(df.iloc[0])
                new_col = df.iloc[2].combine_first(new_col)
                new_col = df.iloc[3].combine_first(new_col)
                new_col = df.iloc[4].combine_first(new_col)
                new_col = df.iloc[5].combine_first(new_col)
                new_col = list(new_col.values)
                
                new_col[0] = "№\nработы"
                new_col[1] = "№ работы"
                new_col[2] = "Наименование раздела, наименование территориального учреждения, другого структурного подразделения Банка России, наименование и местонахождение объекта"
                for i, col_name in enumerate(new_col):
                    if col_name == 'Документ, подтверждающий состояние проекта/работы ':
                        new_col[i] = "№"
                        new_col[i + 1] = "Дата"
                df = df[7:]
                df.columns = new_col
                
            df = df.loc[:, ~df.columns.duplicated()].dropna(axis=1, how='all')

            if year == 2017:
                df_ks_full = df
            else:
                
                df_ks_full = pd.concat([df_ks_full, df],)
                df_ks_full = df_ks_full.reset_index(drop=True)
        
        elif file_type == "KR":
            df = df[4:]
            
            new_col = df.iloc[1].combine_first(df.iloc[0])
            new_col = df.iloc[2].combine_first(new_col)
            
            new_col = list(new_col.values)
            df.columns = new_col
            df = df[4:]
            df = df.loc[:, ~df.columns.duplicated()].dropna(axis=1, how='all')
            print("YEAR: ", year)
            print(df.columns)


            
            if year == 2017:
                df_kr_full = df
            else:
                if year <= 2019:
                    df_kr_full = pd.concat([df_kr_full, df],)
                    df_kr_full = df_ks_full.reset_index(drop=True)
#             print(df_kr_full)



        else:
            pass

In [ ]:
df_ks_full

In [ ]:
df_kr_full

In [ ]:
df_ks_full.to_csv('KS_on_2017_2023.csv', index=False)

## Expenses

In [ ]:
years_dict = {
    2016: '2016',
    2017: '2017',
    2018: '2018',
    2019: '2019',
    2020: '2020_new_base',
    2021: '2021',
}  
df_full = pd.DataFrame()

for year in years_dict:
    path = f"data/Expenses/{years_dict[year]}.xlsx"
    df = pd.read_excel(path)
    df = df[8:]
    row_values = df.iloc[0].tolist()
    row_values[0] = "Итог по смете"
    df.columns = row_values
    df = df[1:]
    
    if year == 2016:
            df_full = df
    else:             
        df_full = pd.concat([df_full, df], ignore_index = True)
        df_full = df_full.reset_index(drop=True)
df_full

In [ ]:
df_full

In [ ]:
df_full.head(20)

In [ ]:
df_full.to_csv('Expenses_2016_2021s.csv', index=False)

In [ ]:
df_expenses = pd.read_csv('Expenses_2016_2021s.csv')
df_expenses

# Data handling and exploration 

## Lists_on


In [4]:
df_lists = pd.read_csv('Lists_on_2016_2022_all_quarters.csv')
df_lists.head()

,"Наименование, адрес, реестровый номер объекта",Год постройки,Дата принятия на баланс Банка России,"Общая площадь объекта недвижимости, \nкв. м",Общая площадь здания кв. м,Занято службами Банка России,Не занято (не исполь\nзуется),Сдается Банком России в аренду,Аренду\nется Банком России,Сдается Банком России в субаренду,...,Передано Банком России по договору безвоз-мездного пользо-вания,Передано Банку России по договору безвоз-мездного пользо-вания,"Балансовая стоимость, руб.","в том числе переоценка, руб.","Сумма начисленной амортизации, руб.","Остаточная стоимость, руб.",Техническое состояние,Год,Квартал,Отделение
0,"Помещение объект № 2 (Вспомогательный), 675000...",1980.0,13.07.1990,217.7,217.7,217.7,0.0,0.0,0.0,0.0,...,0.0,0.0,1638812.21,937438.23,591497.66,1047314.55,Удовлетворительное,2016,1,Благовещенск
1,"Административное здание, 676014, Амурская обла...",1978.0,27.12.1990,389.1,389.1,0.0,0.0,389.1,0.0,0.0,...,0.0,0.0,1050734.76,771306.47,446494.59,604240.17,Удовлетворительное,2016,1,Благовещенск
2,"Административное здание, лит. А, А1, 676980, К...",1959.0,27.12.1990,237.7,237.7,0.0,0.0,0.0,0.0,0.0,...,237.7,0.0,126711.23,53749.02,93108.95,33602.28,Удовлетворительное,2016,1,Благовещенск
3,"Административное здание, 676530, Амурская обла...",1935.0,27.12.1990,830.2,830.2,0.0,830.2,0.0,0.0,0.0,...,0.0,0.0,1657840.58,NaN,1657840.58,NaN,Ветхое,2016,1,Благовещенск
4,"Административное здание, 675000, Амурская обла...",1958.0,14.02.1992,403.9,403.9,403.9,0.0,0.0,0.0,0.0,...,0.0,0.0,303892.63,162432.93,208772.46,95120.17,Удовлетворительное,2016,1,Благовещенск


In [ ]:
grouped_df = df_lists.groupby(['Год', 'Квартал'])['Балансовая стоимость, руб.'].mean()
df_lists['Среднее Балансовая стоимость, руб'] = df_lists.groupby(['Год', 'Квартал'])['Балансовая стоимость, руб.'].transform('mean')
# grouped_df

In [ ]:
import matplotlib.pyplot as plt
grouped_df = grouped_df.reset_index()

# Plot the results
grouped_df['Балансовая стоимость, руб.'].plot()
# plt.xticks([_ for _ in range(1, grouped_df.shape[0] + 1)], [f"Q{i % 4}" for i in range(1, grouped_df.shape[0] + 1)])
# plt.xlabel('Quarter')
# plt.ylabel('Mean Value')
# plt.title('Mean Value by Quarter')
plt.show()

In [ ]:
grouped_df = df_lists.groupby(['Год', 'Квартал'])['Сдается Банком России в аренду'].mean()
df_lists['Среднее Сдается Банком России в аренду'] = df_lists.groupby(['Год', 'Квартал'])['Сдается Банком России в аренду'].transform('mean')
# grouped_df

In [ ]:
# Plot the results
grouped_df.plot()
plt.show()

In [ ]:
df_lists

# Merge data 

In [5]:
df_lists = pd.read_csv('Lists_on_2016_2022_all_quarters.csv')
df_expenses = pd.read_csv('Expenses_2016_2021s.csv')

In [ ]:
df_lists['Наименование, адрес, реестровый номер объекта'] = df_lists['Наименование, адрес, реестровый номер объекта'].str.lower()

In [6]:
df_lists.head(3)

,"Наименование, адрес, реестровый номер объекта",Год постройки,Дата принятия на баланс Банка России,"Общая площадь объекта недвижимости, \nкв. м",Общая площадь здания кв. м,Занято службами Банка России,Не занято (не исполь\nзуется),Сдается Банком России в аренду,Аренду\nется Банком России,Сдается Банком России в субаренду,...,Передано Банком России по договору безвоз-мездного пользо-вания,Передано Банку России по договору безвоз-мездного пользо-вания,"Балансовая стоимость, руб.","в том числе переоценка, руб.","Сумма начисленной амортизации, руб.","Остаточная стоимость, руб.",Техническое состояние,Год,Квартал,Отделение
0,"Помещение объект № 2 (Вспомогательный), 675000...",1980.0,13.07.1990,217.7,217.7,217.7,0.0,0.0,0.0,0.0,...,0.0,0.0,1638812.21,937438.23,591497.66,1047314.55,Удовлетворительное,2016,1,Благовещенск
1,"Административное здание, 676014, Амурская обла...",1978.0,27.12.1990,389.1,389.1,0.0,0.0,389.1,0.0,0.0,...,0.0,0.0,1050734.76,771306.47,446494.59,604240.17,Удовлетворительное,2016,1,Благовещенск
2,"Административное здание, лит. А, А1, 676980, К...",1959.0,27.12.1990,237.7,237.7,0.0,0.0,0.0,0.0,0.0,...,237.7,0.0,126711.23,53749.02,93108.95,33602.28,Удовлетворительное,2016,1,Благовещенск


In [7]:
df_expenses.head(10)

,Итог по смете,Год сметы,Смета,Подразделение БР,Код статьи,Код вида расходов,Наименование вида расходов,Направление расходов,Начальный остаток,Отнесено,Конечный остаток,Дата проведения
0,Итого по смете - Смета Дальневосточного ГУ Бан...,2016,Смета Дальневосточного ГУ Банка России,NaN,NaN,NaN,NaN,NaN,NaN,2.493402e+08,2.493402e+08,NaN
1,Итого по виду - 132.0111.000000,2016,Смета Дальневосточного ГУ Банка России,NaN,132.0111,132.0111.000000,Оплата потребления электроэнергии,NaN,NaN,1.175984e+08,1.175984e+08,NaN
2,Итого по виду - 132.0112.000000,2016,Смета Дальневосточного ГУ Банка России,NaN,132.0112,132.0112.000000,Оплата потребления тепловой энергии,NaN,NaN,1.209013e+08,1.209013e+08,NaN
3,Итого по виду - 132.0113.000000,2016,Смета Дальневосточного ГУ Банка России,NaN,132.0113,132.0113.000000,"Оплата водоснабжения, канализации",NaN,NaN,9.341943e+06,9.341943e+06,NaN
4,Итого по виду - 132.0114.000000,2016,Смета Дальневосточного ГУ Банка России,NaN,132.0114,132.0114.000000,Оплата потребления газа,NaN,NaN,1.498588e+06,1.498588e+06,NaN
5,"Итого по смете - Смета Аппарата,Отделений (Отд...",2016,"Смета Аппарата,Отделений (Отделения-НБ) и РКЦ ДГУ",NaN,NaN,NaN,NaN,NaN,NaN,1.728338e+08,1.728338e+08,NaN
6,Итого по смете - Смета Аппарата и РКЦ Дальнево...,2016,Смета Аппарата и РКЦ Дальневосточного ГУ Банка...,NaN,NaN,NaN,NaN,NaN,NaN,1.628322e+07,1.628322e+07,NaN
7,Итого по смете - Смета Аппарата Дальневосточно...,2016,Смета Аппарата Дальневосточного ГУ Банка Росси...,NaN,NaN,NaN,NaN,NaN,NaN,1.032606e+07,1.032606e+07,NaN
8,Итого по статье - 100.0000,2016,Смета Аппарата Дальневосточного ГУ Банка Росси...,NaN,100.0000,NaN,NaN,NaN,NaN,1.032606e+07,1.032606e+07,NaN
9,Итого по статье - 130.0000,2016,Смета Аппарата Дальневосточного ГУ Банка Росси...,NaN,130.0000,NaN,NaN,NaN,NaN,1.032606e+07,1.032606e+07,NaN


In [8]:
df_expenses = df_expenses.dropna(subset=['Направление расходов'])
df_expenses = df_expenses[df_expenses['Направление расходов'].str.split(" ").apply(len) > 1]


df_expenses['Направление расходов'].isnull().sum(), df_expenses.shape

(0, (23017, 12))

In [9]:
df_final = df_expenses[['Год сметы', 'Наименование вида расходов', 'Направление расходов', 'Отнесено']]

In [10]:
df_final.shape

(23017, 4)

In [11]:
df_final['Год сметы'] = pd.to_datetime(df_final['Год сметы'].astype(str))

/tmp/ipykernel_79327/2091571070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Год сметы'] = pd.to_datetime(df_final['Год сметы'].astype(str))


In [12]:
df_final.isnull().sum()

Год сметы                      0
Наименование вида расходов     0
Направление расходов           0
Отнесено                      16
dtype: int64

In [13]:
df_expenses['Наименование вида расходов'].unique()

array(['Оплата потребления электроэнергии',
       'Оплата потребления тепловой энергии',
       'Оплата водоснабжения, канализации', 'Оплата потребления газа',
       'Расходы на коммунальные услуги',
       'Расходы по оплате потребления электроэнергии',
       'Расходы по оплате всех видов отопления зданий и сооружений (кроме электро- и газового снабжения)',
       'Расходы по оплате услуг водоснабжения, водоотведения',
       'Расходы на коммунальные услуги (расходы прошлых лет)',
       'Расходы по оплате потребления газа',
       'Оплата всех видов отопления зданий и сооружений (кроме электро- и газового снабжения)',
       'Оплата услуг водоснабжения, водоотведения',
       'Коммунальные услуги (расходы прошлых лет)'], dtype=object)

In [15]:
import re

def contains_no_letters(list):
    for s in list:
        if bool(re.search(r'[А-яЁё]+', s)):
            return False
    return True

In [16]:
import itertools

def all_combinations(lst):
    return [list(comb) for i in range(len(lst) + 1, 0, -1) for comb in itertools.combinations(lst, i)]

In [17]:
def check_in_lists_on(all_comb_lst):
    for comb in all_comb_lst:
        combined_mask = df_lists['Наименование, адрес, реестровый номер объекта'].str.contains(comb[0], na=False)
        for j in range(1, len(comb)):
            temp_df = df_lists['Наименование, адрес, реестровый номер объекта'].str.contains(comb[j], na=False)
            combined_mask = combined_mask & temp_df
            
        selected_rows = df_lists[combined_mask]
        
        if len(selected_rows):
            return selected_rows
    return False


In [1]:
### SUPER LONG CALCULATING UNIFIED DATAFRAME FOR LISTS ON & EXPENCES (~35min. cpu)
# names_of_expences_directions = df_expenses['Наименование вида расходов'].unique()
# names_of_expences_directions = np.append(names_of_expences_directions, 'Расходы прошлого года, признанные после отчетной даты.')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Расходы прошлого года.')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Другие расходы.Оплата водоснабжения здание')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоотведения')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Другие расходы.')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата потребления эл.энергии')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоотведение.')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоснабжения.')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Расходы прошлого года')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоснабжения')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата канализации')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата  канализации')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Затраты на горячее водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Затраты на холодное водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Затраты на канализацию')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячего водоснабжения')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата холодного водоснабжения')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячево водоснабжения')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата холодное водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоотведение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячее водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Другие расходы ,')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата холодное водоснабжения')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Холодная вода')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплатата потребления электроэнергии')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячее водоснабжения')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за холодное водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за горячее водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за канализацию')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Плата за технологическое присоед. к электрическим сетям в случаях,не связанных со строительством и кап. ремонтом зданий и сооружений')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Плата за технологическое присоед. к электрическим сетям в случаях,не связанных со строительством и кап. ремонтом зданий и сооружений')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Холодное водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Водоотведение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Горячее водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата потребления электроэнерги')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Другие расходы прошлых лет')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Водоснабжение холодная вода')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Водоснабжение горячая вода')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за потребление тепловой энергии')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Затраты на дизельное топливо')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячего водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата холодного водоснабжение')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за тепловую энергию по тарифам')
# names_of_expences_directions = np.append(names_of_expences_directions, 'по тарифам')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за прочие виды топлиа уголь, дрова и т.д.')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за дизельное топливо')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за электроэнергию')
# names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за водоотведение')
# list_ = [
#     "Оплата за  тепловую энергию",
#     "Оплата за  электроэнергию",
#     "Плата за технологическое присоединение к электрическим сетям",
#     "Оплата за горячее  водоснабжение"
#     "Оплата за  горячее водоснабжение",
#     "Оплата за  водоотведение",
#     "Оплата за услуги водоотведения",
#     "Оплата услуг водоотведения",
#     "Оплата за водоснабжение",
#     "Оплата за услуги горячего водоснабжения",
#     "Оплата за услуги холодного водоснабжения",
#     "Расходы по оплате услуг за холодное водоснабжение",
#     "Хаб.Расходы по оплате коммунальных услуг расходы прошлых лет"
# ]
# for i in list_:
#     names_of_expences_directions = np.append(names_of_expences_directions, i)

# names_of_expences_directions = np.append(names_of_expences_directions, 'по')
# names_of_expences_directions = np.append(names_of_expences_directions, 'за')


# characters = [')', '(']

# df_main = pd.DataFrame()
# expences_col_names = df_expenses.columns
# lists_on_col_names = df_lists.columns

# for index, row in tqdm(df_expenses.iterrows(), total=df_expenses.shape[0]):
#     expences_directions = row['Направление расходов']

    
#     for name in names_of_expences_directions:
#         expences_directions = expences_directions.replace(name, '')
#         expences_directions = expences_directions.lstrip("  ")
#         expences_directions = expences_directions.lstrip(", ")
#         expences_directions = expences_directions.lstrip(". ")
        
    
#     for character in characters:
#         expences_directions = expences_directions.replace(character, '')
#     expences_directions = expences_directions.replace(',', ' ')
#     expences_directions = expences_directions.lower()
    
#     expences_directions_lst = expences_directions.split(" ") 
#     expences_directions_lst = [string for string in expences_directions_lst if string.strip() and string != "-"]
    
#     expences_directions_lst_correct = []
    
#     for word in expences_directions_lst:
#         temp_df = df_lists['Наименование, адрес, реестровый номер объекта'].str.contains(word, na=False)
#         selected_rows = df_lists[temp_df]
#         if len(selected_rows) != 0:
#             expences_directions_lst_correct.append(word)
            
#     expences_directions_lst = expences_directions_lst_correct

#     if len(expences_directions_lst) == 0 or contains_no_letters(expences_directions_lst):
#         continue
#     expences_directions_lst = all_combinations(expences_directions_lst)

#     result = check_in_lists_on(expences_directions_lst)
#     year = row['Год сметы']
    
#     closest_lower_value = result.loc[result['Год'] <= year, 'Год'].max()
#     closest_row = result[result['Год'] == closest_lower_value]
    
#     if len(closest_row) == 0:
#         closest_higher = result[result['Год'] >= year]['Год'].sub(year).abs().idxmin()
#         closest_row = result.loc[[closest_higher]]
#     else:
#         nans_per_row = closest_row.isna().sum(axis=1)

#         # Step 2: Find the index of the row with minimum NaN values
#         min_nans_row_index = nans_per_row.idxmin()

#         # Check if there are multiple minimums
#         min_nans = nans_per_row.min()
#         min_rows_indexes = nans_per_row[nans_per_row == min_nans].index

#         # Step 3: Select the row(s) with minimum NaN values
#         row_with_least_nans = closest_row.loc[min_rows_indexes]
#         closest_row = closest_row.iloc[0]

#     data_dict = {expences_col_names[i]: [row.values[i]] for i in range(len(expences_col_names))}
#     for name in lists_on_col_names:
#         data_dict[name] = closest_row[name]

#     merged_row = pd.DataFrame(data_dict)
#     df_main = pd.concat([df_main, merged_row], ignore_index = True)
    

### Save .csv file

In [ ]:
df_main.to_csv('holy_data.csv', index=False)

# Feature engineering, model trainning and evaluation 

In [153]:
df = pd.read_csv('holy_data.csv')
XX, yy = df[['Наименование вида расходов', 'Направление расходов', "Дата проведения"]], df['Отнесено']
df.head()

,Итог по смете,Год сметы,Смета,Подразделение БР,Код статьи,Код вида расходов,Наименование вида расходов,Направление расходов,Начальный остаток,Отнесено,...,Передано Банком России по договору безвоз-мездного пользо-вания,Передано Банку России по договору безвоз-мездного пользо-вания,"Балансовая стоимость, руб.","в том числе переоценка, руб.","Сумма начисленной амортизации, руб.","Остаточная стоимость, руб.",Техническое состояние,Год,Квартал,Отделение
0,Итого по направлению - Расходы прошлого года. ...,2016,Смета Аппарата Дальневосточного ГУ Банка Росси...,NaN,132.0111,132.0111.000000,Оплата потребления электроэнергии,Расходы прошлого года. Оплата потребления элек...,NaN,9498.58,...,0.0,0.0,1249596.0,NaN,262415.15,987180.85,Удовлетворительное,2016,1,ГУ Банка России
1,NaN,2016,Смета Аппарата Дальневосточного ГУ Банка Росси...,Дальневосточное главное управление Центральног...,132.0111,132.0111.000000,Оплата потребления электроэнергии,Расходы прошлого года. Оплата потребления элек...,NaN,9498.58,...,0.0,0.0,1249596.0,NaN,262415.15,987180.85,Удовлетворительное,2016,1,ГУ Банка России
2,Итого по направлению - Расходы прошлого года. ...,2016,Смета Аппарата Дальневосточного ГУ Банка Росси...,NaN,132.0111,132.0111.000000,Оплата потребления электроэнергии,Расходы прошлого года. Оплата потребления элек...,NaN,16294.01,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2020,4,ГУ Банка России
3,NaN,2016,Смета Аппарата Дальневосточного ГУ Банка Росси...,Дальневосточное главное управление Центральног...,132.0111,132.0111.000000,Оплата потребления электроэнергии,Расходы прошлого года. Оплата потребления элек...,NaN,16294.01,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,2020,4,ГУ Банка России
4,Итого по направлению - Оплата потребления элек...,2016,Смета Аппарата Дальневосточного ГУ Банка Росси...,NaN,132.0111,132.0111.000000,Оплата потребления электроэнергии,Оплата потребления электроэнергии ул.Светланск...,NaN,304853.05,...,0.0,0.0,1249596.0,NaN,321770.96,927825.04,Удовлетворительное,2020,4,ГУ Банка России


## Create feature season for dates

In [154]:
XX['Дата проведения'].fillna('not stated', inplace=True)
seasons = {'01': 'Winter', '02': 'Winter', '03': 'Spring', '04': 'Spring', '05': 'Spring', '06': 'Summer', '07': 'Summer', '08': 'Summer', '09': 'Fall', '10': 'Fall', '11': 'Fall', '12': 'Winter'}
XX['Seasons'] = XX['Дата проведения'].apply(lambda x: seasons.get(x.split('.')[1], x) if x != 'not stated' else x)
XX = XX.drop(columns='Дата проведения')

In [144]:
XX

,Наименование вида расходов,Направление расходов,Seasons
0,Оплата потребления электроэнергии,Расходы прошлого года. Оплата потребления элек...,not stated
1,Оплата потребления электроэнергии,Расходы прошлого года. Оплата потребления элек...,Winter
2,Оплата потребления электроэнергии,Расходы прошлого года. Оплата потребления элек...,not stated
3,Оплата потребления электроэнергии,Расходы прошлого года. Оплата потребления элек...,Winter
4,Оплата потребления электроэнергии,Оплата потребления электроэнергии ул.Светланск...,not stated
...,...,...,...
19836,"Оплата услуг водоснабжения, водоотведения",ЗО 134.1000.03 Столовая Хабаровск. Оплата за х...,Winter
19837,"Оплата услуг водоснабжения, водоотведения",ЗО 134.1000.03 Столовая Хабаровск. Оплата за х...,not stated
19838,"Оплата услуг водоснабжения, водоотведения",ЗО 134.1000.03 Столовая Хабаровск. Оплата за х...,Winter
19839,Коммунальные услуги (расходы прошлых лет),134.1000.05 Столовая Хаб.Расходы по оплате ком...,not stated


In [2]:
yy.fillna(value=yy.median(), inplace=True)

round((XX.isnull().sum()/len(df)*100),2)

NameError: name 'yy' is not defined

In [179]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.model_selection import KFold

X_train, X_test, y_train,  y_test = train_test_split(XX, yy, test_size=0.15, random_state=42)
X_train.shape, X_test.shape

((16864, 3), (2977, 3))

## Hyperparametr tunning + cross validation

In [180]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 50, 1000),
        'depth': trial.suggest_int('depth', 2, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2),
        'loss_function': 'MAPE',
        'verbose': False
    }

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mapes = []

    for train_index, val_index in kf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
#         print(X_train.shape, y_train.shape)
#         print(X_val_fold.shape, y_val_fold.shape)
        model = CatBoostRegressor(**params, task_type="GPU", devices='0:1', cat_features=list(XX.columns))
        model.fit(X_train_fold, y_train_fold, early_stopping_rounds=10, verbose=False)
        
        y_pred = model.predict(X_val_fold)
        mape = mean_absolute_percentage_error(y_val_fold, y_pred)
        mapes.append(mape)

    mapes = np.mean(mapes)
    return mapes  

In [183]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

[I 2023-11-25 11:56:16,287] A new study created in memory with name: no-name-390ca657-9b4e-48d8-a7d8-b6544b8a3dee
[I 2023-11-25 11:57:45,312] Trial 0 finished with value: 1.0309181766212814 and parameters: {'iterations': 928, 'depth': 8, 'learning_rate': 0.183362069357164}. Best is trial 0 with value: 1.0309181766212814.
[I 2023-11-25 12:01:22,326] Trial 1 finished with value: 1.030568753486775 and parameters: {'iterations': 876, 'depth': 10, 'learning_rate': 0.18039544532531457}. Best is trial 1 with value: 1.030568753486775.
[I 2023-11-25 12:03:40,729] Trial 2 finished with value: 1.0304378103168073 and parameters: {'iterations': 912, 'depth': 9, 'learning_rate': 0.18764132566627773}. Best is trial 2 with value: 1.0304378103168073.
[I 2023-11-25 12:04:41,927] Trial 3 finished with value: 1.0318050045803369 and parameters: {'iterations': 935, 'depth': 7, 'learning_rate': 0.05341585664393797}. Best is trial 2 with value: 1.0304378103168073.
[I 2023-11-25 12:05:41,776] Trial 4 finished 

In [184]:
print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

Number of finished trials:  5
Best trial:
Value:  1.0304378103168073
Params: 
    iterations: 912
    depth: 9
    learning_rate: 0.18764132566627773


In [186]:
from matplotlib import pyplot as plt
print(trial.params)
model = CatBoostRegressor(**trial.params,
                          loss_function='MAPE', task_type="GPU", devices='0:1', cat_features=list(XX.columns))  # Or another suitable loss function

# Fit model
model.fit(X_train, y_train, 
          verbose=100,  # Output training progress every 100 iterations
          plot=True)  

{'iterations': 912, 'depth': 9, 'learning_rate': 0.18764132566627773}


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9899092	total: 37.2ms	remaining: 33.9s
100:	learn: 0.9880885	total: 3.15s	remaining: 25.3s
200:	learn: 0.9870045	total: 6.17s	remaining: 21.8s
300:	learn: 0.9864028	total: 9.09s	remaining: 18.5s
400:	learn: 0.9858686	total: 12s	remaining: 15.3s
500:	learn: 0.9854231	total: 14.9s	remaining: 12.2s
600:	learn: 0.9850047	total: 17.9s	remaining: 9.24s
700:	learn: 0.9846125	total: 20.8s	remaining: 6.27s
800:	learn: 0.9842232	total: 23.8s	remaining: 3.3s
900:	learn: 0.9838871	total: 26.8s	remaining: 327ms
911:	learn: 0.9838534	total: 27.1s	remaining: 0us


## Evaluate model result on train data (train in general not in here)

In [187]:
y_pred = model.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred)
1 - mape

-0.1478265193036543

## Get predictions + form submission.csv

In [211]:
df_test = pd.read_csv('test_data.csv')
df_test

,Наименование вида расходов,Направление расходов,Дата платежа,Отнесено
0,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,03 2022,NaN
1,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,04 2022,NaN
2,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,05 2022,NaN
3,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,06 2022,NaN
4,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,07 2022,NaN
...,...,...,...,...
4507,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN
4508,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN
4509,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN
4510,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023,NaN


In [213]:
XX = df_test[['Наименование вида расходов', 'Направление расходов', "Дата платежа"]]
XX


,Наименование вида расходов,Направление расходов,Дата платежа
0,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,03 2022
1,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,04 2022
2,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,05 2022
3,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,06 2022
4,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,07 2022
...,...,...,...
4507,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023
4508,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023
4509,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023
4510,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,01 2023


In [220]:
XX['Дата платежа'].fillna('not stated', inplace=True)
seasons = {'01': 'Winter', '02': 'Winter', '03': 'Spring', '04': 'Spring', '05': 'Spring', '06': 'Summer', '07': 'Summer', '08': 'Summer', '09': 'Fall', '10': 'Fall', '11': 'Fall', '12': 'Winter'}
def month_to_season(month):
    if month in [12, 1, 2]:
        return 'winter'
    elif month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    else:
        return 'fall'
print(type(XX['Дата платежа'][0]))
# create new column based on values of column A
XX['Season'] = XX['Дата платежа'].apply(lambda x: month_to_season(int(x.split('.')[0])) if '.' in x else x)
XX['Seasons'] = XX['Дата платежа'].apply(lambda x: seasons.get(x[:2], x) if x != 'not stated' else x)

XX = XX.drop(columns='Дата платежа')
XX = XX.drop(columns='Season')

XX

<class 'str'>


,Наименование вида расходов,Направление расходов,Seasons
0,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Spring
1,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Spring
2,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Spring
3,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Summer
4,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Summer
...,...,...,...
4507,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,Winter
4508,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,Winter
4509,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,Winter
4510,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,Winter


In [221]:
XX

,Наименование вида расходов,Направление расходов,Seasons
0,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Spring
1,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Spring
2,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Spring
3,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Summer
4,Электроэнергия,134.1000.01 ДГУ. Оплата за электроэнергию по т...,Summer
...,...,...,...
4507,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,Winter
4508,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,Winter
4509,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,Winter
4510,Коммунальные услуги (расходы прошлых лет),134.1000.05 Якутия Оплата потребления газа ул....,Winter


### Use model trinned before 

In [222]:
y_pred = model.predict(XX)
y_pred 

array([3.09606328, 3.09606328, 3.09606328, ..., 3.27124877, 3.27124877,
       3.27124877])

In [223]:
y_pred = pd.DataFrame({'Отнесено': y_pred})
y_pred

,Отнесено
0,3.096063
1,3.096063
2,3.096063
3,3.041944
4,3.041944
...,...
4507,3.271249
4508,3.271249
4509,3.271249
4510,3.271249


### Save submission.csv

In [224]:
y_pred.to_csv("submission.csv", index=False)

# Old idea for submission 

In [3]:
df_test = pd.read_csv('test_data.csv')

df_test

NameError: name 'pd' is not defined

In [126]:
import warnings
warnings.filterwarnings('ignore')
names_of_expences_directions = df_expenses['Наименование вида расходов'].unique()
names_of_expences_directions = np.append(names_of_expences_directions, 'Расходы прошлого года, признанные после отчетной даты.')
names_of_expences_directions = np.append(names_of_expences_directions, 'Расходы прошлого года.')
names_of_expences_directions = np.append(names_of_expences_directions, 'Другие расходы.Оплата водоснабжения здание')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоотведения')
names_of_expences_directions = np.append(names_of_expences_directions, 'Другие расходы.')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата потребления эл.энергии')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоотведение.')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоснабжения.')
names_of_expences_directions = np.append(names_of_expences_directions, 'Расходы прошлого года')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоснабжения')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата канализации')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата  канализации')
names_of_expences_directions = np.append(names_of_expences_directions, 'Затраты на горячее водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Затраты на холодное водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Затраты на канализацию')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячего водоснабжения')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата холодного водоснабжения')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячево водоснабжения')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата холодное водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата водоотведение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячее водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Другие расходы ,')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата холодное водоснабжения')
names_of_expences_directions = np.append(names_of_expences_directions, 'Холодная вода')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплатата потребления электроэнергии')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячее водоснабжения')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за холодное водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за горячее водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за канализацию')
names_of_expences_directions = np.append(names_of_expences_directions, 'Плата за технологическое присоед. к электрическим сетям в случаях,не связанных со строительством и кап. ремонтом зданий и сооружений')
names_of_expences_directions = np.append(names_of_expences_directions, 'Плата за технологическое присоед. к электрическим сетям в случаях,не связанных со строительством и кап. ремонтом зданий и сооружений')
names_of_expences_directions = np.append(names_of_expences_directions, 'Холодное водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Водоотведение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Горячее водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата потребления электроэнерги')
names_of_expences_directions = np.append(names_of_expences_directions, 'Другие расходы прошлых лет')
names_of_expences_directions = np.append(names_of_expences_directions, 'Водоснабжение холодная вода')
names_of_expences_directions = np.append(names_of_expences_directions, 'Водоснабжение горячая вода')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за потребление тепловой энергии')
names_of_expences_directions = np.append(names_of_expences_directions, 'Затраты на дизельное топливо')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата горячего водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата холодного водоснабжение')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за тепловую энергию по тарифам')
names_of_expences_directions = np.append(names_of_expences_directions, 'по тарифам')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за прочие виды топлиа уголь, дрова и т.д.')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за дизельное топливо')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за электроэнергию')
names_of_expences_directions = np.append(names_of_expences_directions, 'Оплата за водоотведение')
list_ = [
    "Оплата за  тепловую энергию",
    "Оплата за  электроэнергию",
    "Плата за технологическое присоединение к электрическим сетям",
    "Оплата за горячее  водоснабжение"
    "Оплата за  горячее водоснабжение",
    "Оплата за  водоотведение",
    "Оплата за услуги водоотведения",
    "Оплата услуг водоотведения",
    "Оплата за водоснабжение",
    "Оплата за услуги горячего водоснабжения",
    "Оплата за услуги холодного водоснабжения",
    "Расходы по оплате услуг за холодное водоснабжение",
    "Хаб.Расходы по оплате коммунальных услуг расходы прошлых лет"
]
for i in list_:
    names_of_expences_directions = np.append(names_of_expences_directions, i)

names_of_expences_directions = np.append(names_of_expences_directions, 'по')
names_of_expences_directions = np.append(names_of_expences_directions, 'за')

print(df_expenses['Наименование вида расходов'].unique())
print(df_test['Наименование вида расходов'].unique())
transform_expenses_dict = {
    'Электроэнергия': 'Оплата потребления электроэнергии',
    'Отопление': 'Оплата всех видов отопления зданий и сооружений (кроме электро- и газового снабжения)',
    'Водоснабжение': 'Оплата услуг водоснабжения, водоотведения' ,
    'Коммунальные услуги (расходы прошлых лет)': 'Расходы на коммунальные услуги (расходы прошлых лет)',
    'Газ': 'Оплата потребления газа'
}
characters = [')', '(']
k = 0
df_main = pd.DataFrame()
expences_col_names = df_expenses.columns
lists_on_col_names = df_lists.columns
mover = 0

for index, row_test in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    ind = index + mover
    if ind < df_test.shape[0]:
        row_test = df_test.loc[ind]
    else:
        break
        
    current_expenses_dir = transform_expenses_dict[row_test['Наименование вида расходов']]
    # print(df_expenses[df_expenses['Направление расходов'] == row_test['Направление расходов']], current_expenses_dir)
    filtered_rows = df_expenses[(df_expenses['Направление расходов'] == row_test['Направление расходов'].replace("  ", " ")) & (df_expenses['Наименование вида расходов'] == current_expenses_dir)]
    n = len(filtered_rows)
    # print(filtered_rows)
    # break
    if n == 0:
        filtered_rows = df_expenses[(df_expenses['Направление расходов'] == row_test['Направление расходов']) & (df_expenses['Наименование вида расходов'] == current_expenses_dir)]
    if n:
        for indexx, row in filtered_rows.iterrows():
            # print(row)
            k += 1
            expences_directions = row['Направление расходов']
    
        
            for name in names_of_expences_directions:
                expences_directions = expences_directions.replace(name, '')
                expences_directions = expences_directions.lstrip("  ")
                expences_directions = expences_directions.lstrip(", ")
                expences_directions = expences_directions.lstrip(". ")
                
            
            for character in characters:
                expences_directions = expences_directions.replace(character, '')
            expences_directions = expences_directions.replace(',', ' ')
            expences_directions = expences_directions.lower()
            
            expences_directions_lst = expences_directions.split(" ") 
            expences_directions_lst = [string for string in expences_directions_lst if string.strip() and string != "-"]
            
            expences_directions_lst_correct = []
        
            for word in expences_directions_lst:
                temp_df = df_lists['Наименование, адрес, реестровый номер объекта'].str.contains(word, na=False)
                selected_rows = df_lists[temp_df]
                if len(selected_rows) != 0:
                    expences_directions_lst_correct.append(word)
                    
            expences_directions_lst = expences_directions_lst_correct
        
            if len(expences_directions_lst) == 0 or contains_no_letters(expences_directions_lst):
                continue
            expences_directions_lst = all_combinations(expences_directions_lst)
        
            result = check_in_lists_on(expences_directions_lst)
            year = row['Год сметы']
            
            closest_lower_value = result.loc[result['Год'] <= year, 'Год'].max()
            closest_row = result[result['Год'] == closest_lower_value]
            
            if len(closest_row) == 0:
                closest_higher = result[result['Год'] >= year]['Год'].sub(year).abs().idxmin()
                closest_row = result.loc[[closest_higher]]
            else:
                nans_per_row = closest_row.isna().sum(axis=1)
    
                min_nans_row_index = nans_per_row.idxmin()
        
                min_nans = nans_per_row.min()
                min_rows_indexes = nans_per_row[nans_per_row == min_nans].index
        
                row_with_least_nans = closest_row.loc[min_rows_indexes]
                closest_row = closest_row.iloc[0]
        
            data_dict = {expences_col_names[i]: [row.values[i]] for i in range(len(expences_col_names))}
            for name in lists_on_col_names:
                data_dict[name] = closest_row[name]
        
            merged_row = pd.DataFrame(data_dict)
            # print(merged_row)
            df_main = pd.concat([df_main, merged_row], ignore_index = True)
    else:
        num_cols = len(df_main.columns)

        # create new row containing NaN values
        new_row = pd.Series([np.nan] * num_cols, index=df_main.columns)
        
        # append new row to dataframe
        df_main = df_main.append(new_row, ignore_index=True)
    mover += n
    

['Оплата потребления электроэнергии' 'Оплата потребления тепловой энергии'
 'Оплата водоснабжения, канализации' 'Оплата потребления газа'
 'Расходы на коммунальные услуги'
 'Расходы по оплате потребления электроэнергии'
 'Расходы по оплате всех видов отопления зданий и сооружений (кроме электро- и газового снабжения)'
 'Расходы по оплате услуг водоснабжения, водоотведения'
 'Расходы на коммунальные услуги (расходы прошлых лет)'
 'Расходы по оплате потребления газа'
 'Оплата всех видов отопления зданий и сооружений (кроме электро- и газового снабжения)'
 'Оплата услуг водоснабжения, водоотведения'
 'Коммунальные услуги (расходы прошлых лет)']
['Электроэнергия' 'Отопление' 'Водоснабжение'
 'Коммунальные услуги (расходы прошлых лет)' 'Газ']


 86%|████████████████████████████████▌     | 3871/4512 [00:36<00:06, 105.60it/s]


In [127]:
df_main.shape

(4249, 39)

In [128]:
round((df_main.isnull().sum()/len(df_main)*100),2)

Итог по смете                                                       98.94
Год сметы                                                           89.72
Смета                                                               89.72
Подразделение БР                                                    90.77
Код статьи                                                          89.72
Код вида расходов                                                   89.72
Наименование вида расходов                                          89.72
Направление расходов                                                89.72
Начальный остаток                                                  100.00
Отнесено                                                            89.72
Конечный остаток                                                    89.72
Дата проведения                                                     90.77
Наименование, адрес, реестровый номер объекта                       89.72
Год постройки                         

In [129]:
mover, k

(641, 641)

In [ ]:
columns_drop = [
    'Итог по смете',
    'Остаточная стоимость, ру',
    'Аренду\nется Банком России.1 ',
    'Сдается Банком России в аренду.1',
    'Не занято (не исполь\nзуется).1 '
]